### Homework 2: Geospatial Analysis EDS 296

In this assignment, you will perform some geospatial visualizations for a region of your choice.
The code constructed should be in Python, and follow a format similar to the tutorials we’ve
been working through in class. Use whatever section headings you like, so long as the tasks
below are included in the notebook!

#### Part 1: 
Using the CMIP6 database hosted on Amazon Web Services, choose any two models you
like: use both their historical simulations and future projections from one of the four major
SSPs (ssp126, ssp245, ssp370, or ssp585). Provide a brief description of the models and
scenarios you chose to include.

#### Let's load the modules that will/might be used

In [102]:
# Load modules
import xarray as xr
import matplotlib.pyplot as plt
import intake
import s3fs
import cartopy.crs as ccrs

ERROR 1: PROJ: proj_create_from_database: Open of /opt/anaconda3/envs/eds296-stevenson/share/proj failed


We need to select two models and two scenarios (one of them being historical) for this exercise. Let's first Open the CMIP6 catalog to view the options that we have for our models

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

In [3]:
# Print the catalog to get a summary of its contents
catalog

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,709
grid_label,10
zstore,522217
dcpp_init_year,60


In [6]:
# We are interested in viewing the different models
catalog.df.source_id.unique()

array(['CMCC-CM2-HR4', 'EC-Earth3P-HR', 'HadGEM3-GC31-MM',
       'HadGEM3-GC31-HM', 'HadGEM3-GC31-LM', 'EC-Earth3P', 'ECMWF-IFS-LR',
       'ECMWF-IFS-HR', 'HadGEM3-GC31-LL', 'CMCC-CM2-VHR4', 'GFDL-CM4',
       'GFDL-AM4', 'IPSL-CM6A-LR', 'E3SM-1-0', 'CNRM-CM6-1', 'GFDL-ESM4',
       'GFDL-CM4C192', 'GFDL-ESM2M', 'GFDL-OM4p5B', 'GISS-E2-1-G',
       'GISS-E2-1-H', 'CNRM-ESM2-1', 'BCC-CSM2-MR', 'BCC-ESM1', 'MIROC6',
       'AWI-CM-1-1-MR', 'EC-Earth3-LR', 'IPSL-CM6A-ATM-HR', 'CESM2',
       'CESM2-WACCM', 'CNRM-CM6-1-HR', 'MRI-ESM2-0', 'CanESM5',
       'SAM0-UNICON', 'GISS-E2-1-G-CC', 'UKESM1-0-LL', 'EC-Earth3',
       'EC-Earth3-Veg', 'FGOALS-f3-L', 'CanESM5-CanOE', 'INM-CM4-8',
       'INM-CM5-0', 'NESM3', 'MPI-ESM-1-2-HAM', 'CAMS-CSM1-0',
       'MPI-ESM1-2-LR', 'MPI-ESM1-2-HR', 'MRI-AGCM3-2-H', 'MRI-AGCM3-2-S',
       'MCM-UA-1-0', 'INM-CM5-H', 'KACE-1-0-G', 'NorESM2-LM',
       'FGOALS-f3-H', 'FGOALS-g3', 'MIROC-ES2L', 'FIO-ESM-2-0', 'NorCPM1',
       'NorESM1-F', 'MPI-ESM1-2-XR'

I am going to use the `AWI-CM-1-1-MR` and `CNRM-CM6-1-HR` models from this list. They are particularly good at simulating sea ice thickness, which is the variable I want to study for this exercise. The two scenarios I will be using are the `historical` and `SSP5-8.5`. I chose the latter because It represents a worst case scenario with the highest emissions. 

`AWI-CM-1-1-MR`: It is considered one of the most advanced `CMIP6` Arctic sea ice models. It has strong representation of ice dyanmics, focuses on the polar region, and has high resolution measurements. 

`CNRM-CM6-1-HR`: Also a high-resolution model that accurately simulates sea ice dyamics. The focus of this model is not on the Arctic like the AWI, rather a global scale. Regardless of this, it is still great for modeling sea ice thickness. 

`Historical`: The historical scenario contains observed data from 1850 to 2014 with variables like greenhouse gases, aerosols, volcanic eruptions, land use change, etc. It is used as a benchmark to evaluate how well models simulate these past observations. 

`SSP5-8.5`: SSP stands for Shared Socioeconomic Pathway. This one in particular represents a worst case scenario, where we continue emissions at the current rate with no concern for reducing. This model is demonstrated from 2015-2100 and is used to determine future projections. 



#### Select Parameters for the model

In [84]:
# Specify search terms to query catalog for the two models
# activity_id: which project do you want? CMIP = historical data, ScenarioMIP = future projections
activity_ids = ['CMIP', 'ScenarioMIP'] 

# source_id: Two models for this exercise 
source_id = ['AWI-CM-1-1-MR', 'NorESM2-LM']

# experiment_id: what experimental configuration do you want? Here we want historical and the four main SSPs
experiment_ids = ['historical', 'ssp585']

# member_id: which ensemble member do you want? Here we want r1i1p1f1
member_id = ['r1i1p1f1']

# table_id: Monthly atmosphere data
table_id = 'SImon' 

# variable_id: Precipitation 
variable_id = 'sithick' 

We have the chosen parameters, let's now search through the catalog and store it as a dataframe

In [85]:
# Search catalog, store results
res = catalog.search(activity_id=activity_ids, 
                     source_id=source_id, 
                     experiment_id=experiment_ids, 
                     member_id=member_id, 
                     table_id=table_id, 
                     variable_id=variable_id)

# Turn results into a dataframe
res = res.df
res

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,SImon,sithick,gn,s3://cmip6-pds/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/hi...,NaN,20181218
1,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp585,r1i1p1f1,SImon,sithick,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/AWI/AWI-CM-1-...,NaN,20181218
2,CMIP,NCC,NorESM2-LM,historical,r1i1p1f1,SImon,sithick,gn,s3://cmip6-pds/CMIP6/CMIP/NCC/NorESM2-LM/histo...,NaN,20190815
3,ScenarioMIP,NCC,NorESM2-LM,ssp585,r1i1p1f1,SImon,sithick,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/NCC/NorESM2-L...,NaN,20191108


#### Choose region of interest
Since I am interested in Sea Ice Thickness, and my models specialize in the poles, I am going to use Alaska as my region of interest. This is a cold region with lots of snow and ice cover. From climate change, we would expect, and have already seen polar caps melting due warming. 

In [86]:
lat_min, lat_max = 50.03, 71.6925
lon_min, lon_max = -172.347846, -132.097822

### Choose two separate time periods, each 30-50 years in length

a. Map the average over each time period separately

b. Map the difference in the averages between the two time periods (note: make sure to label
which time period you subtracted from which!)

For both your sets of maps, display some relevant political/geographic boundaries overlaid
on the region: we saw some examples of how to do this using the Cartopy “feature” toolbox
in the mapping tutorials.

In [88]:
# Store historical AWI data
awi_historical = xr.open_zarr(res['zstore'][0], storage_options = {'anon':True})

# Store SSP585 AWI datta
awi_ssp = xr.open_zarr(res['zstore'][1], storage_options = {'anon':True})

# Store historical NCC data
ncc_historical = xr.open_zarr(res['zstore'][2], storage_options = {'anon':True})

# Store SSP585 NCC data
ncc_ssp = xr.open_zarr(res['zstore'][3], storage_options = {'anon':True})


In [89]:
# Concatenate the data
awi = xr.concat([awi_historical, awi_ssp], dim ='time')

ncc = xr.concat([ncc_historical, ncc_ssp], dim='time')

Now that the data is in different `xarray`'s, we want to splice it into 30-50 year time periods and map it. 

In [96]:
awi['time'] = awi.time.astype('datetime64[ns]')

ncc['time'] = ncc.time.astype('datetime64[ns]')

In [97]:
# Data for early period
awi_early = awi.sel(time=slice("1981-01-01", "2010-12-31"))
ncc_early = ncc.sel(time=slice("1981-01-01", "2010-12-31"))

# Data for later period
awi_late = awi.sel(time=slice("2060-01-01", "2100-12-31"))
ncc_late = ncc.sel(time=slice("2060-01-01", "2100-12-31"))

# Calculate differences
awi_diff = awi_late.mean(dim='time') - awi_early.mean(dim='time')
ncc_diff = ncc_late.mean(dim='time') - ncc_early.mean(dim='time')

### Extract values

In [99]:
# Extract AWI values into a Numpy array
awi_values = awi_diff.sithick.values

# Extract NCC values into a Numpy array
ncc_values = ncc_diff.sithick.values

Check that the units are the same for AWI and NCC

In [100]:
print(awi_early.sithick.units)

print(ncc_early.sithick.units)

m
m


In [116]:
awi_diff

<xarray.Dataset>
Dimensions:   (ncells: 830305, vertices: 16)
Coordinates:
    lat       (ncells) float64 -47.39 -48.48 -49.21 -49.45 ... 59.97 33.46 42.17
    lat_bnds  (ncells, vertices) float64 -47.44 -47.42 -47.39 ... 42.09 42.09
    lon       (ncells) float64 -33.98 -35.53 -36.0 -35.67 ... -42.36 15.88 18.49
    lon_bnds  (ncells, vertices) float64 -33.96 -33.92 -33.91 ... 18.5 18.5 18.5
Dimensions without coordinates: ncells, vertices
Data variables:
    sithick   (ncells) float32 dask.array<chunksize=(830305,), meta=np.ndarray>

Great! They are both in meters.

We need mask our data to the region(Alaska) with the coordinates that we defined earlier. 

In [110]:
# Define logical mask:
# AWI Data
awi_lat = (awi_diff.lat >= lat_min) & (awi_diff.lat <= lat_max)
awi_lon = (awi_diff.lon >= lon_min) & (awi_diff.lon <= lon_max)

# NCC Data
ncc_lat = (ncc_diff.lat >= lat_min) & (ncc_diff.lat <= lat_max)
ncc_lon = (ncc_diff.lon >= lon_min) & (ncc_diff.lon <= lon_max)

AttributeError: 'Dataset' object has no attribute 'lat'

Now let's plot!

In [122]:
# Now use .sel() to subset the data using the boolean masks
awi_subset = awi_diff.sel(
    lat=awi_lat,
    lon=awi_lon
)


KeyError: "no index found for coordinate 'lat'"